# Coursework Part 1: Detecting Spam with Spark

IN432 Big Data coursework 2017, part 1. Classifying messages to detect spam.


## Task a) & b) Read some files and prepare a (f,w) RDD 
a) Start by reading the directory with text files from the distributed file system (e.g. `hdfs://saltdean.nsqdc.city.ac.uk./data/spam/bare/part1`), and loading all text files using wholeTextFiles(), which loads the text per file, i.e. tuples (f,t). (5%)

b) Split the text into words (lower case), creating a (file,word) RDD. (10%)

For both tasks you can use the code from the labs. Don't remove finals 's' (we have already lemmatised data to work with later). 

It is very useful to pack the code into a function that takes a directory as an argument and returns an RDD with (f,w) structure, e.g. `read_fw_RDD`.

Please write two lines of code at the end of the cell that run a little example and print some output. You can comment them out after you have verified that your code works. 

In [ ]:
import re 

dirPath = 'hdfs://saltdean.nsqdc.city.ac.uk./data/spam/bare/part1'

def read_fw_RDD( argDir ): # package tasks a/b into a function for later use
    ... #<<< task a) read the files
    print('Read {} files from directory {}'.format(<no of files>,argDir)) # status message for testing, can be disabled later on
    #print('file word count histogram') # the histogram can be useful for checking later 
    #print(fwL_RDD.map(lambda fwL: (len(fwL[1]))).histogram([0,10,100,1000,10000]))
    ... #<<< task b) split words
    return fw_RDD #,fw_RDD

fw_RDD = read_fw_RDD(dirPath[0:1]) # for testing
print(fw_RDD.take(3)) # for testing

## Task c) Normalised word count lists
Use the code from the labs to generate the \texttt{[(word,count), ...]} list per file and to create a word frequency vector. 

Normalise the term frequency (TF) vector by the total word count per file. (15\%)

This is mostly reusing the lab code. The interesting part here is the normalisation. For normalisation we need to total word count per file. You can use a nested list comprehension for this (go through the (w,c) list and divide each c by the sum of all c, which you can get with a list). 

Another option is to use a separate RDD with (f,twc), where 'twc' is for total word count, and which you can create from the (f,[(w,c), ... ]) RDD. This new RDD can then be joined with the (f,[(w,c), ... ]) RDD and then the (w,c) list be normalised in a list comprehension. 

It would actually be more efficient to do the word counting when you split the words, but that would mix the tasks too much, so please don't do that in your coursework submission.

Again, put your code into a function, and add a short test that can be commented out.

In [ ]:
from operator import add

def reGrpLst(fw_c): # reorganise the tuples
    ... #<<<  
    
def make_f_tfLn_RDD(argDir): 
    fw_RDD = read_fw_RDD( argDir ) # call function from task a & b
    ... #<<< read as in the labs 
    ... #<<< and normalise by 
    return f_wcLn_RDD

f_wcLn_RDD = make_f_tfLn_RDD('hdfs://saltdean/data/spam/bare/part5') # for testing
#print(f_wcLn_RDD.take(1)) # for testing
wcLn = f_wcLn_RDD.take(1)[0][1] # get the first normalised word count list
print(sum([cn for (w,cn) in wcLn])) # the sum of normalised counts should be close to 1 

## Task d) Creating hashed feature vectors 
Use the hashing trick to create fixed size TF vectors. (10%)

Use the code from the week 2 lecture to create the hash vectors.

As before, make it a function and add a short test.

In [ ]:
def hashing_vectorizer(word_count_list, N): 
    # use the code from the lecture

def make_f_wVn_RDD(f_wcLn_RDD, argN):
    # apply hashing_vectorizer
    
N=100
#f_wVn_RDD = make_f_wVn_RDD(make_f_tfLn_RDD(dirPath),N) # for testing
#print(f_wVn_RDD.take(1)[0][1]) # for testing
#print( sum(f_wVn_RDD.take(1)[0][1])) # for testing

## Task e) Create Labeled Points

Determine whether the file is spam (i.e. the filename contains ’spmsg’) and replace the filename by a 1 (spam) or 0 (ham) accordingly. Use map() to create an RDD of LabeledPoint objects. 

See here [http://spark.apache.org/docs/2.0.0/ml-classification-regression.html#logistic-regression](http://spark.apache.org/docs/2.0.0/ml-classification-regression.html#logistic-regression) for an example, and here [http://spark.apache.org/docs/2.0.0/api/python/pyspark.mllib.html#pyspark.mllib.regression.LabeledPoint](http://spark.apache.org/docs/2.0.0/api/python/pyspark.mllib.html#pyspark.mllib.regression.LabeledPoint) for the `LabeledPoint` documentation. (15%)

It's useful to take the RDD with normalised word lists as input. 

For finding the spam messages use `re.search()` see here[https://docs.python.org/3/library/re.html?highlight=re%20search#re.search](https://docs.python.org/3/library/re.html?highlight=re%20search#re.search) for documentation. Search for 'spmsg' in the filename and check whether the result is `None`. The relevan syntax here is <b>`0 if <yourCondition> else 1`</b>, i.e. 0 if 'spmsg' is not in the filename (not spam) and 1 if it is (it's spam).

In [ ]:
from pyspark.mllib.regression import LabeledPoint

def make_lp_RDD(f_tfLn_RDD,argN):
    #<<< make a vector
    #<<< detect spam by filename 
    #<<< transform into LabeledPoint objects
    return lp_RDD

#lp_RDD = make_lp_RNN(make_f_tfLn_RDD('hdfs://saltdean.nsqdc.city.ac.uk./data/spam/bare/part1'),100)
#print(lp_RDD.take(3))

## Task f) Train a classifier 

Use the `LabeledPoint` objects to train the `LogisticRegression` and calculate the accuracy of the model on the training set (again, follow this example [http://spark.apache.org/docs/2.0.0/ml-classification-regression.html#logistic-regression](http://spark.apache.org/docs/2.0.0/ml-classification-regression.html#logistic-regression) and here is the documentation [http://spark.apache.org/docs/2.0.0/api/python/pyspark.mllib.html#pyspark.mllib.classification.LogisticRegressionWithLBFGS](http://spark.apache.org/docs/2.0.0/api/python/pyspark.mllib.html#pyspark.mllib.classification.LogisticRegressionWithLBFGS).  (15%) 

It's useful to start with a normalised word list as input again (because we can later also use it with TF.IDF values).  

In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, NaiveBayes
from pyspark.mllib.util import MLUtils

path = 'hdfs://saltdean/data/spam/stop/part1'

N=100
def trainModel(f_wcL_RDD,N):
    ... #<<< get the training data as LabeledPoint objects.
    print('training data items: {}, correct: {}'.format(trainData.count(), correct)) # output raw numbers
    print('training accuracy {:.1%}'.format(accuracy)) # and accuracy
    return model 

f_wcLn_RDD = make_f_wcLn_RDD(path) # for testing
model = trainModel(f_wcLn_RDD,N) # for testing

## Task g) Test the classifier

Use the files from \texttt{.../data/extra/spam/bare/part10} and prepare them like in task~a)-e) (use the function you created in task e) and before. Then use the trained model to predict the label for each vector you have and compare it to the original to test the performance of your classifier. (10\%) 

In [ ]:
def testModel(model,f_wcL_RDD,N):
    #<<< like with trainModel, transform the data and evaluate it.
    print('test data items: {}, correct:{}'.format(testData.count(),correct))
    print('testing accuracy {:.1%}'.format(accuracy))

testModel(model,make_f_wcLn_RDD('hdfs://saltdean/data/spam/stop/part10'),N) # for testing

## Task h) Run experiments 

Package the whole classifier training and evaluation in one function. Then apply it to the files from `/data/extra/spam/lemm`, `/data/extra/spam/stop` and `/data/extra/spam/lemm_stop` in addition to `/data/extra/spam/bare`  and evaluate the accuracy of your classifier. 

Comment on the effect of *lemmatisation* and *stopword removal* on classification accuracy. Further, evaluate the use of larger training sets and the effect of different vector sizes. Print out the results of your experiments in readable form. (20%) 

You need to create one small fuction that combines tasks f) and g), and then apply it to different datasets sizes, vector sizes, and different preprocessings. 

The combination of the part1-part9 datasets can be achieved by using 'glob' patterns in the filename ('part[1-9]'). This is a feature of the Hadoop filesystem and not well documented in Spark (or anywhere else). You can find a description of its Python implementation here: [https://docs.python.org/3/library/glob.html](https://docs.python.org/3/library/glob.html). You can also supply multiple comma-separated paths, but you'll need to test what works, when you use this feature. Recursive patterns don't seem to work.

Alternatively, you can create unions of RDDs for each part. However, this seems to lead to slower execution. With the latter, it is useful to created arrays of the directory names (part1, ...). When you work with unions, it may be useful to start with an empty RDD. That can be created with `sc.parallelize([])`.

A useful tool for creatng multiple long paths variants is the use of the Python string format() method as used below. There is a good set of example here: [https://docs.python.org/3/library/string.html#format-examples](https://docs.python.org/3/library/string.html#format-examples) and the specification is here: [https://docs.python.org/3/library/string.html#format-specification-mini-language](https://docs.python.org/3/library/string.html#format-specification-mini-language).


In [1]:
# this function combines tasks f) and g)
def trainTestModel(trainPaths,testPath,N):
    ... #<<< just combine training ans testing here
    
# prepare the part directories and the path
dirPattern = 'hdfs://saltdean/data/spam/bare/path[1-{}]' # the {} can be filled by 'dirpath.format(i)' 
# create the path for the test set
testPath = 'hdfs://saltdean/data/spam/bare/path10'

print('EXPERIMENT 1: Testing different training set sizes')
print('Path = {}, N = {}'.format(dirpath,N)) # using format to make sure we record the parameters of the experiment
#<<< make the test set, it will be constant for this experiment
#<<< loop over i the number of parts for training (1-9)
    trainPath = dirPattern.format{i} # in the loop you can create a path like this
    print(trainPath) #just for testing, remove later
    #<<< create the trainRDD (using your make_f_tfLn_RDD method)
    trainTestModel(trainPaths,testPath,N)

print('\nEXPERIMENT 2: Testing different vector sizes')
#<<< loop over different values for N. 3,10,30,100,300, ... is a good pattern
    print('=== N = ',N)
    trainTestModel(trainPaths,testPath,N)

N = 100 # change to what you feel is a good compromise between computation and accuracy
# the dictionary below helps associate description and paths.
setDict = {'No preprocessing':'hdfs://saltdean/data/spam/bare/',
           'Stopwords removed':'hdfs://saltdean/data/spam/stop/',
           'Lemmatised':'hdfs://saltdean/data/spam/lemm/',
           'Lemmatised and stopwords removed':'hdfs://saltdean/data/spam/lemm_stop/'}
print('\nEXPERIMENT 3: Testing differently preprocessed data sets')
print('training on parts 1-9, N = {}'.format(N))
for sp in setDict:
    print('=== ',sp)
    trainPath = setDict[sp]+'part[1-9]'
    #<<< make the test and training data RDD and evaluate 

print('\n====== Done ======')

IndentationError: unexpected indent (<ipython-input-1-abdd88114391>, line 14)

## Task i) (Task for pairs) TF.IDF vectors
You need to address this task if you are working as a pair. 

Calculate the IDF values for each word and generate fixed size TF.IDF vectors for each document (word frequencies still normalised by total document word count). Also evaluate the use of TF.IDF compared to normalised word counts in terms of accuracy. (25%)

To calculate the IDF values you need to create an RDD (w,f) pairs. You can use the function `RDD.distinct()` to remove duplicates and reorganise to create (w,[f, ...]) lists. The length of the list is the document frequency and can be used to calculate the IDF.





In [ ]:
from operator import add
from math import log

trainPath = 'hdfs://saltdean.nsqdc.city.ac.uk./data/spam/lemm_stop/part[1-9]'
testPath = 'hdfs://saltdean.nsqdc.city.ac.uk./data/spam/lemm_stop/part10'

def make_f_wtfiL_RDD(path):
    # Calculuate the IDFs
    fw_RDD = read_fw_RDD(path)
    #<<< keep only unique (f,w) pairs
    #<<< (f,w) -> (w,[f])
    #<<< join the lists of files with reduceByKey
    vocSize = wfL_RDD.count() # calculate the vocabulary size
    print('vocSize: {}'.format(vocSize)) 
    # calculate the IDF values per word by using len() on the list of files
    print('wIdf_RDD.count(): ',wIdf_RDD.count()) # for testing
    print(wIdf_RDD.take(2)) # for testing

    # Get the normalise word counts (TFs) and organise by word (w,(f,cn))
    f_wcLn_RDD = make_f_wcLn_RDD(path) # create the normalised word count lists 
    #print('f_wcLn_RDD: ',f_wcLn_RDD.map(
    #        lambda x: sum([c for (w,c) in x[1]]).histogram([0,10,100,1000,10000]))) # check for the per-file word counts
    #<<<< create a list of tuples [(w,(f,cn)), ..] and use flatmap 
    print('w_fcn_RDD.count(): {}'.format(w_fcn_RDD.count())) # for testing
    print(w_fcn_RDD.take(2)) # for testing

    # now we can join the IFDs and TFs by the words (w,(f,cn)) join (w,idf) to (w,((f,cn),idf))
    #<<< use RDD.join()
    print( 'w_fcnIdf_RDD.count(): ', w_fcnIdf_RDD.count())
    print( w_fcnIdf_RDD.take(2))

    # we have doubly nested tuples (w,((f,cn),idf)) in the RDD, 
    # but they let us calculate the TF.IDF per file and word (f,[(w,cn*idf)]).
    #<<<< map to (f,[(w,cn*idf)])
    print('f_wtfiL_RDD.count()', f_wtfiL_RDD.count())
    print(str(f_wtfiL_RDD.take(2)))

    # with that we can reduce by key (files) to get [(w,tfidf), ...] lists per file.
    #<<< reduceByKey
    print('# of files with TF.IDF vectors: {}'.format(f_wtfiL2_RDD.count()))
    print(f_wtfiL2_RDD.take(2)))

    return f_wtfiL2_RDD


N=100 # choose a value yourself 
print('N: {}, trainPath: {}'.format(N,trainPath))
#<<< you can now apply trainModel and test Model to RDDs created with make_f_wtfiL_RDD()